In [1]:
import numpy as np
from scipy import special as sp
import csv

#need matlab to check
import matlab.engine
eng = matlab.engine.start_matlab()


In [2]:
path = '../domeconduit/Code/TimeDependent/'
eng.addpath (path, nargout= 0 )

In [315]:
# Model parameters
m = {}

Tvec = np.array((950, 900, 850, 800, 750))
m['Ti'] = 3
m['T'] = np.float(Tvec[m['Ti']-1] + 273.15)  # temperature [K]

# densities
m['rho_s']  = 2600.
m['rho_l']  = 2200.
m['rho_hd'] = 2200.
m['rho_cd'] = 2200.

# gas constants
m['Rw'] = 461.5 # J/(kg K)
m['Rc'] = 188.9 # J/(kg K)


# initialise solid fraction curves
fname = 'chis_ti' + str(m['Ti']) + '.csv'
with open(fname, newline='') as f:
    reader = np.array(list(csv.reader(f)),dtype=np.float)

pp = {}
pp['breaks'] = np.array(reader[:,0]).reshape(len(reader),1)
pp['coefs'] = reader[:-1,1:]
m['pf'] = pp





In [336]:
def mat_to_py(var):
  # converts matlab arrays to numpy arrays and makes sure everything is shape Nx1
  val = np.array(var)
  if len(val.shape) < 2:
    val = val[:,np.newaxis]
    L, W = val.shape
    
  if L < W:
    val = val.T
  return val

In [194]:

def viscosity(h2o_d, phi_s_eta, gdot, m):
    """
    Calculates viscosity of silicate melt using 
    - dacite melt viscosity model from Whittington 2009
    - crystal content relative viscosity model from Costa 2005
    
    h2o_d in wt%
    phi_s_eta = phi_s / (phi_s + phi_l)
    gdot is strain rate
    """
    
    h2o_arg = np.log10(h2o_d + 0.26)
    logeta_m = -4.43 + (7618.3 - 17.25*h2o_arg)/(m['T'] - 406.1 + (292.6*h2o_arg))
    eta_m = np.power(10,logeta_m)
    
    if gdot == 0:
        #values from Costa 2005b and 2007b
        kappa   = 0.999916
        phistar = 0.673
        gamma   = 3.98937
        delta   = 16.9386
    else:
        # strain-rate dependent values from Costa 2005 and Caricchi 2007
        phistar = -0.066499*np.tanh(0.913424*np.log10(gdot)+3.850623) + 0.591806
        delta   = -6.301095*np.tanh(0.818496*np.log10(gdot)+2.86)     + 7.462405
        kappa   = -0.000378*np.tanh(1.148101*np.log10(gdot)+3.92)     + 0.999572
        gamma   =  3.987815*np.tanh(0.890800*np.log10(gdot)+3.24)     + 5.099645
        
    pih = np.sqrt(np.pi)
    B   = 2.5
    
    phifr = phi_s_eta/phistar
    
    top = (1. + np.power(phifr,delta))    #numerator
    efarg = 0.5*pih*phifr/kappa*(1. + np.power(phifr,gamma))  #error function argument
    eta_phi = top/np.power((1. - kappa*sp.erf(efarg)), B*phistar)

    eta = eta_m*eta_phi
        
    return eta, eta_m, eta_phi



In [26]:
print(np.log10(gdot), eng.log10(gdot))

-5.0 -5.0


In [202]:
h2o_d = 5.
phi_s_eta = 0.5
gdot = 1.0e-5

eta1, eta_m1, eta_phi1 = viscosity(h2o_d, phi_s_eta, gdot, m)
eta2, eta_m2, eta_phi2 = eng.tdcFV('calc_eta', h2o_d*0.01, phi_s_eta, m['T'], 2, gdot, nargout=3)

print(eta1, eta_m1, eta_phi1)
print(eta2, eta_m2, eta_phi2)



233815.67501426488 5832.851929387548 40.08599529781234
233815.67501426488 5832.851929387548 40.08599529781234


In [203]:
def solubility(p, mh, m):
    """
    calculate dissolved h2o and co2 using pressure, temperature, mole fraction water
    using constitutive relations from Liu et al 2005
    Be careful with units!
    
    p    pressure in Pa
    mh   mole fraction water (range 0 to 1)
    m    dictionary of values
    
    h2o_d   dissolved h2o in wt%
    co2_d   dissolved co2 in ppm
    
    """
    
    # convert p to MPa
    p = 1e-6*p
    
    T = m['T']
    
    # Precompute for speed.
    Pw      = p*mh
    Pc      = p*(1-mh)
    sqrtPw  = np.sqrt(Pw)
    Pw15    = Pw*sqrtPw # Pw^1.5

    # These equations require MPa, Kelvin, and return wt% and ppm in weight. Assuming saturation!
    h2o_d = (354.94*sqrtPw + 9.623*Pw - 1.5223*Pw15)/T + 0.0012439*Pw15 + Pc*(-1.084e-4*sqrtPw - 1.362e-5*Pw)
    co2_d = Pc*(5668 - 55.99*Pw)/T + Pc*(0.4133*sqrtPw + 2.041e-3*Pw15)
    
    return h2o_d, co2_d
    

In [212]:
p = 100.e6
mh = 0.9

h2o_d1, co2_d1 = solubility(p, mh, m)
h2o_d2, co2_d2 = eng.tdcFV('solubility_liu_explicit', p, m['T'], mh, nargout=2)

print(h2o_d1, co2_d1)
print(h2o_d2, co2_d2)

3.6514262684227448 62.23487438438163
3.6514262684227448 62.23487438438163


In [429]:
def solid_mass_frac(pMPA, pp):
    """
    Calculate solid mass fraction for specified pressure using piecewise polynomials 
    Pressure in MPa here
    
    NB the shapes of these may become a problem. 
    NEEDS TO BE CHECKED!!!
    """    
    
    # check where the the pressure is between the two breakpoints
    ppbreak = pp['breaks'].transpose()
    tmp = np.all([pMPA>=ppbreak[0,:-1], pMPA<ppbreak[0,1:]], axis=0)
    
    # gather index of polynomial for each pressure in pMPA
    chi_ind = mat_to_py(np.argmax(tmp, axis=1))
    
    # make dp and coef matrices of size pMPA. Can't figure out how to make the coeffs in one line
    dp  = pMPA - pp['breaks'][chi_ind,0]
    cf0 = pp['coefs'][chi_ind,0]
    cf1 = pp['coefs'][chi_ind,1]
    cf2 = pp['coefs'][chi_ind,2]
    cf3 = pp['coefs'][chi_ind,3]
    
    # calculate piecewise polynomial
    chi_s = cf0*np.power(dp,3) + cf1*np.power(dp,2) + cf2*dp + cf3
    
    # check at low pressure, chi_s = 1
    chi_s[pMPA<pp['breaks'][ 0]] = pp['coefs'][0,3]
    
    # check that there are no negative values at extremely high pressures
    chi_s[chi_s<0] = 0
    
    return chi_s

In [326]:
def density (p, phi_g, mh, m):
    """
    calculate magma density given pressure, porosity and mole fraction water
    """
    
    # get dissolved volatiles
    h2o_d, co2_d = solubility_liu(p, mh, m)
    
    # convert to mass fractions
    h2o_d = 1e-2*h2o_d
    co2_d = 1e-6*co2_d
    
    c1  = 1/(1 - h2o_d - co2_d)
    c2  = 1/(1 + h2o_d*c1*(m['rho_l']/m['rho_hd']) + co2_d*c1*(m['rho_l']/m['rho_cd']))
    c12 = c1*c2
    
    chi_s = solid_mass_frac(p*1e-6, m['pf'])
    
    phi_s = chi_s*m['rho_l']*c12*(1-phi_g)/(m['rho_s'] + chi_s*(c12*m['rho_l']-m['rho_s']))
    phi_s_eta = phi_s/(1-phi_g)
    
    phi_l = (1 - phi_s - phi_g)*c2 
    rho_g = p*(mh/(m['Rw']*m['T']) + (1-mh)/(m['Rc']*m['T']))
    
    rho = m['rho_s']*phi_s + m['rho_l']*phi_l*c1 + rho_g*phi_g
    
    return rho
    

In [430]:
p = mat_to_py(np.array([100e6,50e6]))

chi_s = solid_mass_frac(1e-6*p, m['pf'])
print(chi_s)

# matlab values:
# 0.477586596754590   0.581054531506694


[[0.4775866 ]
 [0.58105453]]


In [427]:


p = mat_to_py(np.array([1.054280000e8,  1.052828331167970e8]))
phi_g = mat_to_py(np.array([0.070420583176296,0.070739299110460]))
mh = mat_to_py(np.array([0.934573912110104,0.934790009940851]))

rho = density(p, phi_g, mh, m)
print(rho)

# matlab values:
#  2219.934768390909 2219.305166750766

[[2219.93476839]
 [2219.30516675]]


In [428]:
p = np.linspace(1e6,100e6,num=101)
p = mat_to_py(p)

print(p.shape)
mh = 0.9

# need to reshape pressure

pMPa = p*1e-6

h2o_d, co2_d = solubility_liu(p, mh, m)
print(h2o_d.shape)

chi_s = solid_mass_frac(pMPa, m['pf'])

rho = density(p, 0.1, mh, m)

print(chi_s.shape)

(101, 1)
(101, 1)
(101, 1)
